In [1]:
%load_ext autoreload
%autoreload 2
from inxss.utils_spectrum import calc_Sqw_from_Syy_Szz
from inxss.experiment import SimulatedExperiment

import torch
import numpy as np
from scipy.interpolate import RegularGridInterpolator

from inxss import SpectrumDataset, SpecNeuralRepr, Particle, PsiMask

import matplotlib.pyplot as plt

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/global/homes/z/zhantao/.conda/envs/sqt/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = torch.load('/pscratch/sd/z/zhantao/inxs_steering/SpinW_data/summarized_AFM_data_2023Aug01.pt')
print(data.keys())

dict_keys(['Syy', 'Szz', 'S', 'w_grid', 'q_grid', 'params'])


In [3]:
model = SpecNeuralRepr.load_from_checkpoint(
    '/pscratch/sd/z/zhantao/inxs_steering/lightning_logs/version_13505970/checkpoints/last.ckpt'
    )

/global/homes/z/zhantao/.conda/envs/sqt/lib/python3.9/site-packages/pytorch_lightning/utilities/migration/utils.py:49: PossibleUserWarning: The loaded checkpoint was produced with Lightning v2.0.6, which is newer than your current Lightning version: v1.9.0
  rank_zero_warn(


In [4]:
prior_configs = {'types': ['uniform', 'uniform'], 'args': [{'low': 20, 'high': 40}, {'low': -5, 'high': 5}]}

particle = Particle(1000, 2, prior_configs=prior_configs)

In [5]:
map_path = '/pscratch/sd/z/zhantao/inxs_steering/La2NiO4_bool'
psi_mask = PsiMask(map_path, scale_factor=0.5, device='cpu', preload=False)

h_grid
k_grid
l_grid
w_grid


In [ ]:
%timeit psi_mask.get_mask(1)